# Credit Risk Resampling Techniques

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [4]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,next_pymnt_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,...,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,0.0,0.0,8.0,0.0,1609.0,14.0,w,10204.87,10204.87,355.29,355.29,295.13,60.16,0.0,0.0,0.0,375.35,May-2019,0.0,1.0,Individual,0.0,178.0,38199.0,3.0,4.0,2.0,4.0,3.0,36590.0,...,1.0,0.0,4.0,6.0,4775.0,447.0,77.7,0.0,0.0,93.0,96.0,3.0,3.0,0.0,20.0,1.0,0.0,1.0,2.0,1.0,1.0,10.0,3.0,3.0,2.0,8.0,0.0,0.0,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,0.0,0.0,17.0,1.0,18368.0,34.0,w,24293.13,24293.13,873.53,873.53,706.87,166.66,0.0,0.0,0.0,929.09,May-2019,0.0,1.0,Individual,0.0,165.0,228119.0,3.0,3.0,2.0,3.0,4.0,42273.0,...,4.0,9.0,3.0,13.0,13419.0,29502.0,28.4,0.0,0.0,87.0,286.0,1.0,1.0,2.0,1.0,11.0,3.0,7.0,11.0,8.0,13.0,14.0,13.0,18.0,11.0,17.0,0.0,0.0,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,0.0,0.0,8.0,0.0,13247.0,21.0,w,19636.79,19636.79,485.44,485.44,363.21,122.23,0.0,0.0,0.0,529.88,May-2019,0.0,1.0,Individual,0.0,0.0,45684.0,2.0,3.0,2.0,3.0,4.0,32437.0,...,0.0,4.0,1.0,6.0,5711.0,2839.0,62.1,0.0,0.0,159.0,156.0,7.0,4.0,0.0,7.0,7.0,6.0,2.0,5.0,2.0,7.0,8.0,5.0,13.0,5.0,8.0,0.0,0.0,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,0.0,1.0,10.0,1.0,17996.0,23.0,w,9719.34,9719.34,335.33,335.33,280.66,54.67,0.0,0.0,0.0,353.55,May-2019,0.0,1.0,Individual,0.0,0.0,68784.0,0.0,4.0,1.0,3.0,9.0,50788.0,...,3.0,5.0,4.0,4.0,6878.0,4899.0,75.1,0.0,0.0,100.0,89.0,24.0,9.0,0.0,34.0,0.0,0.0,4.0,6.0,4.0,4.0,17.0,6.0,6.0,6.0,10.0,0.0,0.0,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,0.0,1.0,14.0,0.0,9091.0,28.0,w,21614.73,21614.73,484.36,484.36,385.27,99.09,0.0,0.0,0.0,520.39,May-2019,0.0,1.0,Individual,0.0,0.0,177745.0,0.0,1.0,0.0,2.0,14.0,16828.0,...,2.0,0.0,2.0,4.0,13673.0,19699.0,28.6,0.0,0.0,67.0,138.0,11.0,11.0,4.0,11.0,0.0,0.0,4.0,6.0,7.0,11.0,6.0,12.0,18.0,6.0,14.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
df.columns

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

# Split the Data into Training and Testing

In [7]:
# Encoding the columns with pd.get_dummies() to numerical data 
encoding_col= pd.get_dummies(df, columns = ['home_ownership', 'verification_status', 'pymnt_plan', 'hardship_flag', 'debt_settlement_flag', 'initial_list_status','application_type'])
encoding_col.head()
encoding_col.columns


Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'issue_d',
       'loan_status', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'total_acc', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal',
       'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt',
       'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op',
       'mo_sin_rcnt_tl', 'mort_acc', 'mt

In [8]:
# # Create our target
y = encoding_col['loan_status']

# Create our features
X = encoding_col.drop(['loan_status','issue_d', 'next_pymnt_d'], axis ='columns')


In [9]:
# Feature selection and cleaning up the data 
from sklearn.feature_selection import SelectKBest, chi2

bestfeatures = SelectKBest(score_func=chi2, k=25)
fit = bestfeatures.fit(X, y)
dfscores= pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
# concat 2 DF for better visuals 
featureScores = pd.concat([dfcolumns,dfscores], axis=1)
featureScores.columns = ['Specs', 'Score']
print(featureScores.nlargest(25, 'Score'))

                         Specs          Score
72             tot_hi_cred_lim  391414.330426
25                 tot_cur_bal  382581.112655
3                   annual_inc  131817.543726
74              total_bc_limit  113924.642110
42                 avg_cur_bal   49703.180494
35                  max_bal_bc   47389.597630
13                 total_pymnt   32146.476636
14             total_pymnt_inv   32131.523994
31                total_bal_il   32012.638588
16               total_rec_int   21810.314981
73           total_bal_ex_mort   16356.508433
15             total_rec_prncp   13643.384156
17          total_rec_late_fee   11399.643306
37            total_rev_hi_lim   11096.386284
43              bc_open_to_buy    9299.404516
75  total_il_high_credit_limit    9204.960862
9                    revol_bal    4246.982023
0                    loan_amnt    3283.521741
11                   out_prncp    1209.762619
12               out_prncp_inv    1205.846405
46                 delinq_amnt    

In [10]:
# selecting the features based off of the SelectKBest the top 16 out of 25 
X1 = encoding_col[['tot_hi_cred_lim', 'tot_cur_bal','annual_inc', 'total_bc_limit', 'avg_cur_bal',\
                   'max_bal_bc', 'total_pymnt', 'total_pymnt_inv', 'total_bal_il',\
                   'total_rec_int', 'total_bal_ex_mort', 'total_rec_prncp', 'total_rec_late_fee',\
                   'total_rev_hi_lim', 'bc_open_to_buy', 'total_il_high_credit_limit']]

In [11]:
X1.dtypes

tot_hi_cred_lim               float64
tot_cur_bal                   float64
annual_inc                    float64
total_bc_limit                float64
avg_cur_bal                   float64
max_bal_bc                    float64
total_pymnt                   float64
total_pymnt_inv               float64
total_bal_il                  float64
total_rec_int                 float64
total_bal_ex_mort             float64
total_rec_prncp               float64
total_rec_late_fee            float64
total_rev_hi_lim              float64
bc_open_to_buy                float64
total_il_high_credit_limit    float64
dtype: object

In [12]:
X1.shape

(68817, 16)

In [13]:
X1.describe()

,tot_hi_cred_lim,tot_cur_bal,annual_inc,total_bc_limit,avg_cur_bal,max_bal_bc,total_pymnt,total_pymnt_inv,total_bal_il,total_rec_int,total_bal_ex_mort,total_rec_prncp,total_rec_late_fee,total_rev_hi_lim,bc_open_to_buy,total_il_high_credit_limit
count,6.881700e+04,6.881700e+04,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04
mean,2.100332e+05,1.634976e+05,8.821371e+04,29734.128558,14142.936193,6297.491710,970.639473,970.431151,4.324044e+04,311.855687,6.133843e+04,658.748750,0.035072,42431.895360,16991.852667,5.572240e+04
std,1.928088e+05,1.723699e+05,1.155800e+05,26795.394232,15863.878993,5702.281757,1229.563301,1229.569864,4.956130e+04,279.786954,5.738798e+04,1126.237377,1.086720,37166.229316,20447.276820,5.095845e+04
min,3.600000e+03,2.350000e+02,4.000000e+01,100.000000,20.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000,2.350000e+02,0.000000,0.000000,200.000000,0.000000,1.270000e+02
25%,6.697700e+04,3.570000e+04,5.000000e+04,11600.000000,3505.000000,2558.000000,403.760000,403.550000,1.422300e+04,115.440000,2.650300e+04,250.370000,0.000000,19200.000000,3644.000000,2.288000e+04
50%,1.467100e+05,9.960600e+04,7.300000e+04,22100.000000,8261.000000,4874.000000,710.790000,710.790000,2.862600e+04,225.250000,4.535700e+04,449.890000,0.000000,32800.000000,10239.000000,4.200000e+04
75%,3.036400e+05,2.472060e+05,1.040000e+05,39300.000000,19859.000000,8321.000000,1214.600000,1214.590000,5.406500e+04,415.710000,7.657000e+04,762.170000,0.000000,54100.000000,22806.000000,7.249900e+04
max,3.292782e+06,3.140082e+06,8.797500e+06,509400.000000,448583.000000,207484.000000,41418.981165,41418.980000,1.260281e+06,2735.400000,1.295455e+06,40000.000000,75.240000,759500.000000,506507.000000,1.426964e+06


In [14]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y, random_state=1)
X_train.shape
y_test.shape
# X_test.shape

(17205,)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [16]:
# Importing RandomOverSampler
from imblearn.over_sampling import RandomOverSampler


In [17]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resample = ros.fit_resample(X_train, y_train)

In [18]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resample)


LogisticRegression(random_state=1)

In [19]:
# making predictions 
predictions = model.predict(X_test)

In [20]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, predictions)

0.5834785331900822

In [21]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, predictions)

In [22]:
# Print the imbalanced classification report
report = classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

   high_risk       0.01      0.48      0.02       101
    low_risk       1.00      0.69      0.82     17104

    accuracy                           0.69     17205
   macro avg       0.50      0.58      0.42     17205
weighted avg       0.99      0.69      0.81     17205



### SMOTE Oversampling

In [23]:
from imblearn.over_sampling import SMOTE

In [24]:
# Resample the training data with SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)

In [25]:
# Counting to verify the classes are now equal in size 
Counter(y_resampled)

Counter({'high_risk': 51366, 'low_risk': 51366})

In [26]:
# Train the Logistic Regression model using the resampled data
model= LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)

In [27]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)



0.6048501190156435

In [28]:
# regular 'overall' accuracy 
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred) 

0.6742226097064806

In [29]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   54,    47],
       [ 5558, 11546]])

In [30]:
# Print the imbalanced classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.01      0.53      0.02       101
    low_risk       1.00      0.68      0.80     17104

    accuracy                           0.67     17205
   macro avg       0.50      0.60      0.41     17205
weighted avg       0.99      0.67      0.80     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [32]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 246, 'low_risk': 246})

In [33]:
# Train the Logistic Regression model using the resampled data
modelcc = LogisticRegression(solver='lbfgs', random_state=1)
modelcc.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [38]:
# Creating predictions and display the confusion matrix 
predictions = modelcc.predict(X_test)
confusion_matrix(y_test, predictions)

array([[  53,   48],
       [7846, 9258]])

In [39]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions)

0.5330146847706286

In [40]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.52      0.54      0.01      0.53      0.28       101
   low_risk       0.99      0.54      0.52      0.70      0.53      0.28     17104

avg / total       0.99      0.54      0.52      0.70      0.53      0.28     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [45]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_SM_resampled, y_SM_resampled = smote_enn.fit_resample(X_train, y_train)
Counter(y_train)

Counter({'high_risk': 246, 'low_risk': 51366})

In [46]:
# Train the Logistic Regression model using the resampled data
modelSM =LogisticRegression(solver='lbfgs', random_state=1)
modelSM.fit(X_SM_resampled, y_SM_resampled)

LogisticRegression(random_state=1)

In [47]:
# Creating predictions and displaying the confusion matrix
predict_SM = modelSM.predict(X_test)
confusion_matrix(y_test, predict_SM)

array([[  67,   34],
       [7241, 9863]])

In [48]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predict_SM)

0.6200075368855875

In [49]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predict_SM))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.66      0.58      0.02      0.62      0.39       101
   low_risk       1.00      0.58      0.66      0.73      0.62      0.38     17104

avg / total       0.99      0.58      0.66      0.73      0.62      0.38     17205

